# Reddit Scraper für vergangene Daten 

## Import 

In [102]:
import praw
import pandas as pd
from datetime import datetime, timedelta, timezone
import os
import psaw as ps
from dotenv import load_dotenv
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import torch
from tqdm import tqdm
from psaw import PushshiftAPI

In [103]:
# Lade die .env-Datei
dotenv_loaded = load_dotenv("zugang_reddit.env")  # Falls die Datei anders heißt, anpassen
# Prüfe, ob die Datei geladen wurde
print(f".env geladen? {dotenv_loaded}")


.env geladen? True


In [104]:
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent=os.getenv("USER_AGENT")
)

print("Reddit API erfolgreich verbunden!")


Reddit API erfolgreich verbunden!


In [105]:
for post in reddit.subreddit("CryptoCurrency").search("Bitcoin", sort="new", limit=5):
    print(post.title, post.created_utc)


The fact we have continual focus on price and such shows 99% only care about a profit. Is that true for everyone? 1739657923.0
"Looking at the chart you can see Bitcoin if forming the historical inverted tree house pattern" 1739655608.0
Goldman Sachs Doubles Down on Bitcoin ETFs, Increases Holdings by 120% in Q4 1739648709.0
Banking Giant Barclays Owns $136,834,631 Worth of BlackRock’s Bitcoin Exchange-Traded Fund 1739645324.0
Abu Dhabi’s Sovereign Fund Invests $436 Million in BlackRock’s Bitcoin ETF 1739642215.0


Cryptos und Subreddits 

In [106]:
crypto_terms = {
    # 🔹 Top Coins
    "Ethereum": ["ethereum", "eth", "ether", "ethereum 2.0", "eth 2.0"],
    "Wrapped Ethereum": ["wrapped ethereum", "weth"],
    "Solana": ["solana", "sol", "sol coin"],
    "Avalanche": ["avalanche", "avax"],
    "Polkadot": ["polkadot", "dot"],
    "Near Protocol": ["near protocol", "near"],
    "Polygon": ["polygon", "matic"],
    "XRP": ["xrp", "ripple"],
    "Cardano": ["cardano", "ada"],
    "Cronos": ["cronos", "cro"],
    "Vulcan Forged PYR": ["vulcan forged", "pyr"],
    "Chiliz": ["chiliz", "chz"],
    "Illuvium": ["illuvium", "ilv"],
    "Ronin": ["ronin", "ron"],
    "Band Protocol": ["band protocol", "band"],
    "Optimism": ["optimism", "op"],
    "Celestia": ["celestia", "tia"],
    "Numerai": ["numerai", "nmr"],
    "Aethir": ["aethir", "ath"],
    "Sui": ["sui"],
    "Hyperliquid": ["hyperliquid", "hyp"],
    "Robinhood Coin": ["robinhood", "hood"],
    "Trump Coin": ["trump coin"],
    "USD Coin": ["usd coin", "usdc"],
    "Binance Coin": ["binance", "bnb"],
    "Litecoin": ["litecoin", "ltc"],
    "Dogecoin": ["dogecoin", "doge"],
    "Tron": ["tron", "trx"],
    "Aave": ["aave"],
    "Hedera": ["hedera", "hbar"],
    "Filecoin": ["filecoin", "fil"],
    "Cosmos": ["cosmos", "atom"],
    "Gala": ["gala"],
    "The Sandbox": ["sandbox", "sand"],
    "Audius": ["audius", "audio"],
    "Render": ["render", "rndr"],
    "Kusama": ["kusama", "ksm"],
    "VeChain": ["vechain", "vet"],
    "Chainlink": ["chainlink", "link"],
    "Berachain": ["berachain", "bera"],
    "TestCoin": ["testcoin", "test"],

    # 🔹 Meme-Coins
    "Shiba Inu": ["shiba inu", "shib"],
    "Pepe": ["pepe"],
    "Floki Inu": ["floki inu", "floki"],
    "Bonk": ["bonk"],
    "Wojak": ["wojak"],
    "Mog Coin": ["mog"],
    "Doge Killer (Leash)": ["leash"],
    "Baby Doge Coin": ["baby doge", "babydoge"],
    "Degen": ["degen"],
    "Toshi": ["toshi"],
    "Fartcoin": ["fartcoin"],
    "Banana": ["banana"],
    "Kabosu": ["kabosu"],
    "Husky": ["husky"],
    "Samoyedcoin": ["samoyedcoin", "samo"],
    "Milkbag": ["milkbag"],

    # 🔹 New Coins
    "Arbitrum": ["arbitrum", "arb"],
    "Starknet": ["starknet", "strk"],
    "Injective Protocol": ["injective", "inj"],
    "Sei Network": ["sei"],
    "Aptos": ["aptos", "apt"],
    "EigenLayer": ["eigenlayer", "eigen"],
    "Mantle": ["mantle", "mnt"],
    "Immutable X": ["immutable x", "imx"],
    "Ondo Finance": ["ondo"],
    "Worldcoin": ["worldcoin", "wld"],
    "Aerodrome": ["aerodrome", "aero"],
    "Jupiter": ["jupiter", "jup"],
    "THORChain": ["thorchain", "rune"],
    "Pendle": ["pendle"],
    "Kujira": ["kujira", "kuji"],
    "Noble": ["noble"],
    "Stride": ["stride", "strd"],
    "Dymension": ["dymension", "dym"],
    "Seamless Protocol": ["seamless", "seam"],
    "Blast": ["blast"],
    "Merlin": ["merlin"],
    "Tapioca": ["tapioca"],
    "Arcadia Finance": ["arcadia"],
    "Notcoin": ["notcoin", "not"],
    "Omni Network": ["omni"],
    "LayerZero": ["layerzero", "lz"],
    "ZetaChain": ["zetachain", "zeta"],
    "Friend.tech": ["friendtech"]
}


In [107]:
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance",        # Diskussionen über die Binance-Plattform  
    "Bitcoin",
    "ethtrader"
]

## Scraping 

Scraping Funktionen

In [108]:
# 🔹 GPU-Nutzung prüfen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Verwende Gerät: {device}")

# 🔹 CryptoBERT-Modell laden
MODEL_NAME = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)
model.eval()  # Setzt das Modell in den Evaluationsmodus

# 🔹 Effiziente Batch-Sentiment-Analyse
def analyze_sentiment_batch(texts, batch_size=32):
    """Effiziente GPU-gestützte Sentiment-Analyse mit CryptoBERT für eine Liste von Texten."""
    results = []

    texts = [t if isinstance(t, str) and t.strip() != "" else "neutral" for t in texts]

    for i in tqdm(range(0, len(texts), batch_size), desc="🔍 Analysiere Sentiments"):
        batch_texts = texts[i : i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, max_length=512, padding=True).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        scores = F.softmax(outputs.logits, dim=1)
        labels = ["bearish", "neutral", "bullish"]

        for i in range(len(batch_texts)):
            sentiment = labels[torch.argmax(scores[i]).item()]
            confidence = scores[i].max().item()
            results.append((sentiment, confidence))

    return results

# 🔹 **Kombinierte Funktion: Normalisierung + Sentiment**
def normalize_and_analyze_sentiment(df_posts, df_comments, comment_threshold=500):
    print("🚀 Starte Normalisierung & Sentiment-Analyse...")

    # 🗑️ **1️⃣ Duplikate entfernen**
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])

    # 🔍 **2️⃣ Fehlende Werte auffüllen**
    for col in ["selftext", "title"]:
        if col in df_posts.columns:
            df_posts[col] = df_posts[col].fillna("")
        if col in df_comments.columns:
            df_comments[col] = df_comments[col].fillna("")

    # 📝 **3️⃣ Full Text für NLP & Sentiment-Analyse**
    df_posts["full_text"] = (df_posts["title"] + " " + df_posts["selftext"]).str.strip()
    df_comments["full_text"] = df_comments["selftext"].str.strip()

    # 🏷️ **4️⃣ Datum & Uhrzeit normalisieren**
    for df in [df_posts, df_comments]:
        df["date"] = pd.to_datetime(df["created_utc"], errors="coerce").dt.date
        df["time"] = pd.to_datetime(df["created_utc"], errors="coerce").dt.time

    # 🚀 **5️⃣ Spalten-Typen konvertieren**
    df_posts["score"] = df_posts["score"].astype(int, errors="ignore")
    df_comments["score"] = df_comments["score"].astype(int, errors="ignore")

    # 🚫 **6️⃣ Spam-Nutzer mit zu vielen Kommentaren entfernen**
    if "author" in df_comments.columns:
        frequent_users = df_comments["author"].value_counts()
        frequent_users = frequent_users[frequent_users > comment_threshold].index
        df_comments = df_comments[~df_comments["author"].isin(frequent_users)]

    # ✂️ **7️⃣ Redundante Spalten entfernen**
    drop_columns_posts = ["created_utc", "num_comments"] if "num_comments" in df_posts.columns else ["created_utc"]
    drop_columns_comments = ["created_utc"]
    df_posts = df_posts.drop(columns=drop_columns_posts, errors="ignore")
    df_comments = df_comments.drop(columns=drop_columns_comments, errors="ignore")

    # 🔄 **8️⃣ Spalten-Reihenfolge anpassen**
    df_posts = df_posts[["post_id", "subreddit", "crypto", "search_term", "title", "selftext", "full_text", "author", "score", "date", "time"]]
    df_comments = df_comments[["comment_id", "post_id", "author", "score", "selftext", "full_text", "date", "time"]]

    # 🔹 **9️⃣ Sentiment-Analyse für Posts**
    tqdm.pandas()
    df_posts[["sentiment", "sentiment_confidence"]] = pd.DataFrame(
        analyze_sentiment_batch(df_posts["full_text"].tolist()), index=df_posts.index
    )

    # 🔹 **🔟 Sentiment-Analyse für Kommentare**
    df_comments[["sentiment", "sentiment_confidence"]] = pd.DataFrame(
        analyze_sentiment_batch(df_comments["selftext"].tolist()), index=df_comments.index
    )

    # 🔥 Spalte `full_text` entfernen, da sie nicht mehr benötigt wird
    df_posts = df_posts.drop(columns=["full_text"], errors="ignore")
    df_comments = df_comments.drop(columns=["full_text"], errors="ignore")

    # 🔍 Debugging: Überprüfen, ob `full_text` entfernt wurde
    print(f"📌 Spalten in df_posts_clean: {df_posts_clean.columns.tolist()}")
    print(f"📌 Spalten in df_comments_clean: {df_comments_clean.columns.tolist()}")


    print(f"✅ Normalisierung & Sentiment abgeschlossen: {df_posts.shape[0]} Posts, {df_comments.shape[0]} Kommentare.")
    return df_posts, df_comments


🚀 Verwende Gerät: cuda


In [109]:
start_of_period = datetime(2024, 11, 1, tzinfo=timezone.utc)  # Startdatum (1. November 2024)
now = datetime.now(timezone.utc)  # Enddatum (Jetzt)

print(f"🚀 Starte Scraping-Prozess für {start_of_period.strftime('%Y-%m-%d')} bis {now.strftime('%Y-%m-%d')}...")

# 🔹 Scraper ausführen
df_posts, df_comments = scrape_reddit(start_of_period, now)

# 🔹 Daten normalisieren und Sentiment-Analyse durchführen
df_posts_clean, df_comments_clean = normalize_and_analyze_sentiment(df_posts, df_comments)


🚀 Starte Scraping-Prozess für 2024-11-01 bis 2025-02-15...

🔎 Scraping r/CryptoCurrency...
🔍 Suche nach: ethereum in r/CryptoCurrency...
✅ Post gefunden: Tipping system like Ko-fi or Buy Me a Coffee but with crypto ? (Suchbegriff: ethereum)
✅ Post gefunden: What Big Companies Are Building on Ethereum (Suchbegriff: ethereum)
✅ Post gefunden: Tired of the same limitations on Ethereum? There's a “Movement” on the horizon! (Suchbegriff: ethereum)
✅ Post gefunden: Solana Apps Generate 10x More Revenue Than Ethereum: Research (Suchbegriff: ethereum)
✅ Post gefunden: Bitcoin ETFs maintain market lead as Ethereum ETFs see surge in institutional adoption, 13F filings show (Suchbegriff: ethereum)
✅ Post gefunden: XRP: Fomo or Truth? (Suchbegriff: ethereum)
✅ Post gefunden: Ethereum Pectra Upgrade Confirmed for April 2025 with Fusaka Next in Line (Suchbegriff: ethereum)
✅ Post gefunden: Why Vitalik Buterin’s ‘make communism great again’ quip triggered Ethereum investors (Suchbegriff: ethereum)
✅ 

🔍 Analysiere Sentiments: 100%|██████████| 8785/8785 [31:24<00:00,  4.66it/s]  


📌 Spalten in df_posts_clean: ['crypto', 'search_term', 'subreddit', 'post_id', 'title', 'selftext', 'author', 'score', 'full_text', 'sentiment', 'sentiment_confidence', 'comment_id', 'type', 'date', 'time']
📌 Spalten in df_comments_clean: ['post_id', 'comment_id', 'author', 'score', 'selftext', 'sentiment', 'sentiment_confidence', 'type', 'date', 'time', 'title', 'num_comments', 'crypto', 'search_term', 'full_text']
✅ Normalisierung & Sentiment abgeschlossen: 7641 Posts, 281101 Kommentare.


In [112]:
df_comments_clean.head()

,comment_id,post_id,author,score,selftext,date,time,sentiment,sentiment_confidence,title,num_comments,type
0,mcxeou9,1iq5mon,fistfucker07,3,Not sure how active it is yet. But flare (FLR...,2025-02-15,17:07:04,neutral,0.662513,None,None,comment
1,mcxfioh,1iq5mon,you_can_choose,1,Barteos$,2025-02-15,17:11:22,bearish,0.881935,None,None,comment
2,mcxngpw,1iq5mon,Zestyclose-Power-132,1,I think you could look at discord/telegram bot...,2025-02-15,17:51:14,neutral,0.596018,None,None,comment
3,mcxv670,1iq5mon,HSuke,1,> I know I could simply give my wallet address...,2025-02-15,18:29:46,neutral,0.512703,None,None,comment
4,mcxiyw6,1iq5mon,blaziken8x,1,Kraken just came out with something that's lik...,2025-02-15,17:28:44,bullish,0.624669,None,None,comment


In [113]:
df_posts_clean.head()

,post_id,subreddit,crypto,search_term,title,selftext,author,score,date,time,sentiment,sentiment_confidence,comment_id,type
0,1iq5mon,CryptoCurrency,Ethereum,ethereum,Tipping system like Ko-fi or Buy Me a Coffee b...,Hi\n\nI'm looking for a way to accept small ti...,Nayko93,11,2025-02-15,16:54:53,neutral,0.633046,None,post
1,1iq4hwf,CryptoCurrency,Ethereum,ethereum,What Big Companies Are Building on Ethereum,,MinimalGravitas,11,2025-02-15,16:03:13,bullish,0.514681,None,post
2,1iq2xh0,CryptoCurrency,Ethereum,ethereum,Tired of the same limitations on Ethereum? The...,Are you a blockchain developer or enthusiast l...,NoahCJ,0,2025-02-15,14:50:15,bullish,0.749526,None,post
3,1iq1v1t,CryptoCurrency,Ethereum,ethereum,Solana Apps Generate 10x More Revenue Than Eth...,,Afonsoo99,0,2025-02-15,13:57:02,neutral,0.529202,None,post
4,1ipm5nu,CryptoCurrency,Ethereum,ethereum,Bitcoin ETFs maintain market lead as Ethereum ...,,partymsl,22,2025-02-14,21:58:03,neutral,0.809498,None,post


## Merge

In [115]:
# 🔹 Einheitliche Spaltenstruktur für den Merge herstellen
df_posts_clean["comment_id"] = None  # Posts haben keine `comment_id`
df_comments_clean["crypto"] = None  # Kommentare haben kein `crypto`
df_comments_clean["search_term"] = None  # Kommentare haben keinen `search_term`
df_comments_clean["subreddit"] = None  # Kommentare haben keine `subreddit`
df_comments_clean["title"] = None  # Kommentare haben keinen Titel
df_comments_clean["num_comments"] = None  # Kommentare haben keine Kommentaranzahl

# 🔹 `type`-Spalte für Unterscheidung hinzufügen
df_posts_clean["type"] = "post"
df_comments_clean["type"] = "comment"

# 🔹 Gemeinsame Spalten definieren
common_columns = [
    "post_id", "comment_id", "subreddit", "author", "score",
    "selftext", "title", "sentiment", "sentiment_confidence",
    "crypto", "search_term", "date", "time", "type"
]

# 🔹 Merging der Daten (Posts + Kommentare)
df_merged = pd.concat(
    [df_posts_clean[common_columns], df_comments_clean[common_columns]], 
    ignore_index=True
)

# 🔍 Debugging: Überprüfung der Größe
print(f"📌 Merged Dataset: {df_merged.shape[0]} Einträge (Posts + Kommentare)")

# 🔹 Überprüfen, ob alles korrekt normalisiert wurde
df_merged.head()


📌 Merged Dataset: 288742 Einträge (Posts + Kommentare)


,post_id,comment_id,subreddit,author,score,selftext,title,sentiment,sentiment_confidence,crypto,search_term,date,time,type
0,1iq5mon,None,CryptoCurrency,Nayko93,11,Hi\n\nI'm looking for a way to accept small ti...,Tipping system like Ko-fi or Buy Me a Coffee b...,neutral,0.633046,Ethereum,ethereum,2025-02-15,16:54:53,post
1,1iq4hwf,None,CryptoCurrency,MinimalGravitas,11,,What Big Companies Are Building on Ethereum,bullish,0.514681,Ethereum,ethereum,2025-02-15,16:03:13,post
2,1iq2xh0,None,CryptoCurrency,NoahCJ,0,Are you a blockchain developer or enthusiast l...,Tired of the same limitations on Ethereum? The...,bullish,0.749526,Ethereum,ethereum,2025-02-15,14:50:15,post
3,1iq1v1t,None,CryptoCurrency,Afonsoo99,0,,Solana Apps Generate 10x More Revenue Than Eth...,neutral,0.529202,Ethereum,ethereum,2025-02-15,13:57:02,post
4,1ipm5nu,None,CryptoCurrency,partymsl,22,,Bitcoin ETFs maintain market lead as Ethereum ...,neutral,0.809498,Ethereum,ethereum,2025-02-14,21:58:03,post


## Export 

In [116]:
# Setze den Pfad zu deinem Google Drive Ordner
DRIVE_PATH = "G:/Meine Ablage/reddit/"
POSTS_CSV = os.path.join(DRIVE_PATH, "reddit_posts.csv")
COMMENTS_CSV = os.path.join(DRIVE_PATH, "reddit_comments.csv")
MERGED_CSV = os.path.join(DRIVE_PATH, "reddit_merged.csv")

Funktion zum Export 

In [117]:
def append_to_csv(df_new, filename, key_column):
    """Hängt neue Daten an eine bestehende CSV an & entfernt Duplikate."""
    file_path = os.path.join(DRIVE_PATH, filename)

    try:
        # Falls Datei existiert, alte Daten einlesen
        if os.path.exists(file_path):
            df_existing = pd.read_csv(file_path, sep="|", encoding="utf-8-sig", on_bad_lines="skip")
            
            # 🔹 Daten zusammenführen & Duplikate nach `key_column` entfernen (neuere Werte behalten)
            df_combined = pd.concat([df_existing, df_new], ignore_index=True).drop_duplicates(subset=[key_column], keep="last")
        else:
            df_combined = df_new  # Falls keine Datei existiert, neue Daten direkt nutzen

        # 🔹 CSV speichern
        df_combined.to_csv(
            file_path,
            index=False,
            sep="|",
            encoding="utf-8-sig",
            lineterminator="\n"
        )
        print(f"✅ Datei erfolgreich aktualisiert: {file_path}")

    except Exception as e:
        print(f"Fehler beim Speichern der Datei {filename}: {e}")

def export_to_drive(df_posts, df_comments, df_merged):
    """Speichert Posts, Kommentare & die gemergte Datei mit Duplikat-Prüfung."""
    try:
        append_to_csv(df_posts, "reddit_posts.csv", key_column="post_id")
        append_to_csv(df_comments, "reddit_comments.csv", key_column="comment_id")
        append_to_csv(df_merged, "reddit_merged.csv", key_column="comment_id")  # Falls Kommentare entscheidend sind

    except Exception as e:
        print(f"Fehler beim Export: {e}")

In [118]:
# 🔹 Export-Funktion aufrufen
export_to_drive(df_posts_clean, df_comments_clean, df_merged)

✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit/reddit_posts.csv
✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit/reddit_comments.csv
✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit/reddit_merged.csv


In [119]:
df_merged.head()

,post_id,comment_id,subreddit,author,score,selftext,title,sentiment,sentiment_confidence,crypto,search_term,date,time,type
0,1iq5mon,None,CryptoCurrency,Nayko93,11,Hi\n\nI'm looking for a way to accept small ti...,Tipping system like Ko-fi or Buy Me a Coffee b...,neutral,0.633046,Ethereum,ethereum,2025-02-15,16:54:53,post
1,1iq4hwf,None,CryptoCurrency,MinimalGravitas,11,,What Big Companies Are Building on Ethereum,bullish,0.514681,Ethereum,ethereum,2025-02-15,16:03:13,post
2,1iq2xh0,None,CryptoCurrency,NoahCJ,0,Are you a blockchain developer or enthusiast l...,Tired of the same limitations on Ethereum? The...,bullish,0.749526,Ethereum,ethereum,2025-02-15,14:50:15,post
3,1iq1v1t,None,CryptoCurrency,Afonsoo99,0,,Solana Apps Generate 10x More Revenue Than Eth...,neutral,0.529202,Ethereum,ethereum,2025-02-15,13:57:02,post
4,1ipm5nu,None,CryptoCurrency,partymsl,22,,Bitcoin ETFs maintain market lead as Ethereum ...,neutral,0.809498,Ethereum,ethereum,2025-02-14,21:58:03,post
